# C8: Evaluate The Performance of Deep Learning Models

## 8.1 Data Split

### 8.1.1 Split automatically when fit model

在fit model的时候添加validation_split参数来设定validation set占比，然后训练的时候就会在打印出验证集的验证结果

```
model.fit(X, Y, validation_split=0.33, nb_epoch=150, batch_size=10)
```
example:

In [1]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np

# set random seed
seed = 7
np.random.seed(seed)

# load data
dataset = np.loadtxt("./data_set/pima-indians-diabetes.data", delimiter=',')
X = dataset[:, 0:8]
Y = dataset[:, 8]

# create model
model = Sequential()
model.add(Dense(12, input_dim=8, init='uniform', activation='relu'))
model.add(Dense(8, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='sigmoid'))

# compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit model
model.fit(X, Y, validation_split=0.33, nb_epoch=150, batch_size=10)


Train on 514 samples, validate on 254 samples
Epoch 1/150
514/514 [==============================] - 0s - loss: 0.6902 - acc: 0.5661 - val_loss: 0.6814 - val_acc: 0.6732
Epoch 2/150
514/514 [==============================] - 0s - loss: 0.6773 - acc: 0.6401 - val_loss: 0.6620 - val_acc: 0.6732
Epoch 3/150
514/514 [==============================] - 0s - loss: 0.6661 - acc: 0.6401 - val_loss: 0.6519 - val_acc: 0.6732
Epoch 4/150
514/514 [==============================] - 0s - loss: 0.6591 - acc: 0.6401 - val_loss: 0.6513 - val_acc: 0.6732
Epoch 5/150
514/514 [==============================] - 0s - loss: 0.6568 - acc: 0.6420 - val_loss: 0.6462 - val_acc: 0.6732
Epoch 6/150
514/514 [==============================] - 0s - loss: 0.6452 - acc: 0.6381 - val_loss: 0.6386 - val_acc: 0.6850
Epoch 7/150
514/514 [==============================] - 0s - loss: 0.6349 - acc: 0.6634 - val_loss: 0.6496 - val_acc: 0.6457
Epoch 8/150
514/514 [==============================] - 0s - loss: 0.6390 - acc: 0.6673

Using TensorFlow backend.


### 8.1.2 Split manually

在训练之前手动将数据集分成训练集和验证集，这里推荐sklearn.model_selection里的 train_test_split()函数

```
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.33, random_state=seed)
```

然后在fit model里的时候设定validation_data=(X_val, Y_val)

```
model.fit(X_train, Y_train, validation_data=(X_val, Y_val), nb_epoch=150, batch_size=10)
```

例子：

In [2]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
from sklearn.model_selection import train_test_split

# set random seed
seed = 7
np.random.seed(seed)

# load data
dataset = np.loadtxt("./data_set/pima-indians-diabetes.data", delimiter=',')
X = dataset[:, 0:8]
Y = dataset[:, 8]

# split data
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.33, random_state=seed)

# create model
model = Sequential()
model.add(Dense(12, input_dim=8, init='uniform', activation='relu'))
model.add(Dense(8, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='sigmoid'))

# compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit model
model.fit(X_train, Y_train, validation_data=(X_val, Y_val), nb_epoch=150, batch_size=10)

# evaluate model
scores = model.evaluate(X_val, Y_val, verbose=0)
print model.metrics_names
print 'val loss:', scores[0], 'val acc:', scores[0]

Train on 514 samples, validate on 254 samples
Epoch 1/150
514/514 [==============================] - 0s - loss: 0.6903 - acc: 0.6109 - val_loss: 0.6845 - val_acc: 0.6378
Epoch 2/150
514/514 [==============================] - 0s - loss: 0.6733 - acc: 0.6576 - val_loss: 0.6728 - val_acc: 0.6378
Epoch 3/150
514/514 [==============================] - 0s - loss: 0.6611 - acc: 0.6576 - val_loss: 0.6647 - val_acc: 0.6378
Epoch 4/150
514/514 [==============================] - 0s - loss: 0.6565 - acc: 0.6576 - val_loss: 0.6596 - val_acc: 0.6378
Epoch 5/150
514/514 [==============================] - 0s - loss: 0.6519 - acc: 0.6576 - val_loss: 0.6523 - val_acc: 0.6378
Epoch 6/150
514/514 [==============================] - 0s - loss: 0.6417 - acc: 0.6615 - val_loss: 0.6417 - val_acc: 0.6614
Epoch 7/150
514/514 [==============================] - 0s - loss: 0.6377 - acc: 0.6654 - val_loss: 0.6325 - val_acc: 0.6299
Epoch 8/150
514/514 [==============================] - 0s - loss: 0.6265 - acc: 0.6654

### 8.3 Manual k-Fold Cross Validation

Use sklearn.model_selection.StratifiedKFold Class to split the dataset into 10 folds

```
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
for train_idx, val_idx in kfold.split(X, Y)
    X_train = X[train_idx]
    Y_train = Y[train_idx]
```

In [15]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import StratifiedKFold
import numpy as np

seed = 7
np.random.seed(seed)

dataset = np.loadtxt("./data_set/pima-indians-diabetes.data", delimiter=',')
X = dataset[:, 0:8]
Y = dataset[:, 8]

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []

for train_idx, val_idx in kfold.split(X, Y):
    X_train = X[train_idx]
    Y_train = Y[train_idx]
    X_val = X[val_idx]
    Y_val = Y[val_idx]
    
    model = Sequential()
    model.add(Dense(12, input_dim=8, init='uniform', activation='relu'))
    model.add(Dense(8, init='uniform', activation='relu'))
    model.add(Dense(1, init='uniform', activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, Y_train, nb_epoch=150, batch_size=10, verbose=0)
    scores = model.evaluate(X_val, Y_val, verbose=0)
    
    print("%s: %.2f%%"%(model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1]*100)

print("%.2f%% (+/- %.2f%%)"%(np.mean(cvscores), np.std(cvscores)))


acc: 71.43%
acc: 70.13%
acc: 74.03%
acc: 79.22%
acc: 75.32%
acc: 71.43%
acc: 74.03%
acc: 68.83%
acc: 68.42%
acc: 73.68%
72.65% (+/- 3.11%)
